# Tarea Almacenamiento y captura de datos

> Integrantes: Thamara López, Fabián Ortega

Realice un proceso de web scraping para el sitio https://jetsmart.com/cl/es/. Para ello realice las siguientes tareas:  

1) Utilizar Selenium para emular un navegador y cargar la página requerida (1 punto).  
2) Buscar vuelos desde Santiago a Arica del 1 de febrero al 28 de febrero utilizando los campos de inputs del buscador de la página (1 punto).
3) Una vez en la página de resultados, extraiga la información de precios y fechas de vuelos de ida y de vuelta para todos los días de febrero y genere un dataframe con la información de precios y fechas (2 puntos).  
4) Crear una base de datos sqlite con la información del dataframe (1 punto).  
5) Realizar una consulta SQL para seleccionar los vuelos de ida y vuelta con el precio total mínimo (1 punto).  

Utilizando la API de Twitter, realice las siguientes tareas:

1) Extraer los últimos 1.000 tweets de tu usuario preferido que escriba en español (1 punto).  
2) Hacer un análisis de frecuencia de palabras (100 palabras más frecuentes), habiendo removido primero stopwords y usando tokenización en español (2 puntos).  
3) Extraer 100 usuarios con perfil público que son seguidos por su usuario de preferencia y extraer los últimos 10 tweets de cada uno de esos usuarios (1 punto).  
4) Hacer un análisis de frecuencia de palabras igual al de la parte b) (100 palabras más frecuentes), de los tweets extraídos en la parte c) (2 puntos).   




Cada pregunta tiene una escala de 1 a 7. El promedio de ambas preguntas resulta en la nota final de la tarea. 

In [70]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
import sqlite3 as sql

Funciones auxiliares para simplificar el scraping

In [3]:
def ClickOn(xpath, error=True, time_sleep=2):
    '''Función para hacer click según xpath'''
    try: 
        WebDriverWait(driver, 5)\
            .until(EC.element_to_be_clickable((By.XPATH,
                                            xpath)))\
            .click()
        time.sleep(time_sleep)
    except:
        if error == True:
            raise('Error')
        else:
            pass

def SendKey(xpath, key,error=True, time_sleep=2):
    '''Función para enviar keys a input  según xpath'''
    try: 
        WebDriverWait(driver, 5)\
            .until(EC.element_to_be_clickable((By.XPATH,
                                            xpath)))\
            .send_keys(key)
        time.sleep(time_sleep)
    except:
        if error == True:
            raise('Error')
        else:
            pass

def Find(xpath):
    txt = driver.find_element_by_xpath(xpath)

    return txt.text
    

Creamos driver

In [19]:
options =  webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
driver.implicitly_wait(3)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/fabianalexis/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/l4/br6xdwsn3wj7ntybwy6zwmvh0000gn/T/ipykernel_87194/2803746617.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [11]:
# Xpath del botón de cierre del modal que aparece al entrar en la página
btn_close = '/html/body/div[1]/main/div[7]/div/div[6]/div/div/div[1]/button'
# Página jetsmart
driver.get('https://jetsmart.com/cl/es/')
time.sleep(5)
ClickOn(btn_close, error=False) # función para hacer click por xpath

In [40]:
# Xpaths
origen = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[1]/input[1]'
scl = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[2]/div[3]/ul/li[1]'
destino = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[1]/input[2]'
ar = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/route-selector/div[2]/div[3]/ul/li[2]'
next_month = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/date-selector/div[3]/div[2]/div/div[1]/span[2]'
fecha_inicio = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/date-selector/div[3]/div[2]/div/div[2]/div/div[2]/div[2]/span[2]'
search = '/html/body/div[1]/main/div[2]/searchbox/div/div/div[1]/div/form/div[2]/button'

# Funciones para indicar destinto y fecha de vuelos.
ClickOn(origen)
ClickOn(scl)
ClickOn(destino)
ClickOn(ar)
ClickOn(next_month)
ClickOn(fecha_inicio)
ClickOn(fecha_inicio, error=False)
ClickOn(search)

Generamos código que scrapea en página de vuelos obteniendo el precio de ida y vuelta hasta el 28 de febrero

In [54]:
fecha = '/html/body/app/div/div/div[2]/div/section/div/dummy-flight/ac-flight-page/div/form[1]/div[1]/section/div[1]/ac-flight-fee-option-list/div[2]/header/div/div[1]/label/span'
precio_ida = '/html/body/app/div/div/div[2]/div/section/div/dummy-flight/ac-flight-page/div/form[1]/div[1]/section/div[1]/ac-flight-fee-option-list/div[2]/ul/ac-flight-fee-option[1]/li/div/div/div/div/div[2]/div[2]/ac-flight-fare/div/div[1]/div'
next_btn = '/html/body/app/div/div/div[2]/div/section/div/dummy-flight/ac-flight-page/div/form[1]/div[1]/section/div[1]/ac-flight-fee-option-list/div[1]/div[2]/i'
precio_vuelta = '/html/body/app/div/div/div[2]/div/section/div/dummy-flight/ac-flight-page/div/form[1]/div[1]/section/div[2]/ac-flight-fee-option-list/div[2]/ul/ac-flight-fee-option/li/div/div/div/div/div[2]/div[2]/ac-flight-fare/div/div[1]'
p_ida = []
d_ida = []
p_vuelta = []

i = 0
while i < 28:
    # Precios de ida
    price_ida = Find(precio_ida).replace('$ ','').replace('.', '')
    date_ida = re.sub(".* ", "",Find(fecha))
    print(date_ida)
    print(price_ida)
    p_ida.append(price_ida)
    d_ida.append(date_ida)

    # Precios de vuelta
    price_vuelta = Find(precio_vuelta).replace('$ ','').replace('.', '')
    print(price_vuelta)
    p_vuelta.append(price_vuelta)

    ClickOn(next_btn, time_sleep=4)
    i = int(date_ida.replace('-02',""))

01-02
99000
25000
02-02
52000
25000
03-02
52000
46000
04-02
46000
34000
05-02
34000
34000
06-02
39000
39000
07-02
39000
34000
08-02
34000
25000
09-02
34000
29000
10-02
52000
46000
11-02
39000
29000
12-02
34000
34000
13-02
25000
46000
14-02
29000
46000
15-02
34000
52000
16-02
25000
39000
17-02
46000
39000
18-02
25000
29000
19-02
29000
39000
20-02
25000
46000
21-02
25000
39000
22-02
25000
34000
23-02
25000
29000
24-02
39000
39000
25-02
25000
29000
26-02
29000
29000
27-02
25000
29000
28-02
25000
29000


Generamos dataframe con los datos scrapeados

In [157]:
viaje = pd.DataFrame({'fecha_ida':d_ida, 'precio_ida':p_ida,
'fecha_vuelta':d_ida, 'precio_vuelta':p_vuelta})
# Separo fecha en día y mes
viaje['dia'] = viaje.fecha_ida.str.split("-", expand=True)[0]
viaje['mes'] = viaje.fecha_ida.str.split("-", expand=True)[1]
viaje.head()

,fecha_ida,precio_ida,fecha_vuelta,precio_vuelta,dia,mes
0,01-02,99000,01-02,25000,01,02
1,02-02,52000,02-02,25000,02,02
2,03-02,52000,03-02,46000,03,02
3,04-02,46000,04-02,34000,04,02
4,05-02,34000,05-02,34000,05,02


In [158]:
viaje = viaje.drop(['fecha_ida', 'fecha_vuelta'], axis=1)

In [160]:
viaje.dia = viaje.dia.map(int)
viaje.mes = viaje.mes.map(int)
viaje.head()

,precio_ida,precio_vuelta,dia,mes
0,99000,25000,1,2
1,52000,25000,2,2
2,52000,46000,3,2
3,46000,34000,4,2
4,34000,34000,5,2


### Creamos base de datos sqlite

In [161]:
conn = sql.connect('data/viajes.db')
viaje.to_sql('viajes', conn)

In [162]:
cur = conn.cursor()
cur.execute("SELECT dia, mes, precio_ida, precio_vuelta FROM  viajes")

In [163]:
rows = cur.fetchall()
rows[0:10]

[(1, 2, '99000', '25000'),
 (2, 2, '52000', '25000'),
 (3, 2, '52000', '46000'),
 (4, 2, '46000', '34000'),
 (5, 2, '34000', '34000'),
 (6, 2, '39000', '39000'),
 (7, 2, '39000', '34000'),
 (8, 2, '34000', '25000'),
 (9, 2, '34000', '29000'),
 (10, 2, '52000', '46000')]

Obtengo precio de ida más bajo

In [164]:
pd.read_sql('''SELECT  dia, MIN(precio_ida) as precio_ida_minimo
FROM    viajes''', conn)
## La fecha de ida con el precio minimo es el 13-02, el cual es de 25.000 clp

,dia,precio_ida_minimo
0,13,25000


Obtengo precio de vuelta más bajo según fecha de viaje de ida (viaje de vuelta no puede ser antes del día 13)

In [167]:
pd.read_sql('''SELECT  dia, MIN(precio_vuelta) as precio_ida_minimo
FROM    viajes
WHERE dia >= 13''', conn)

,dia,precio_ida_minimo
0,18,29000


R: El precio de ida y vuelta más barato es de 54.000 clp